In [1]:
import os,sys, math, ROOT
from array import array
import numpy as np
# %jsroot on

#Importing a helper file where some functions are defined
from helper import *

#Input file
inputfile = ROOT.TFile("../Data/MC_signal.root","READ")

#Output file
outputfile = ROOT.TFile("output.root","RECREATE")

#Accessing the "TTree" from root that contains the data (one entry per event)
tree = inputfile.Get("ntuplizer/tree")

nentries = tree.GetEntries()
print("Number of entries: ", nentries)

Welcome to JupyROOT 6.26/06
Number of entries:  568864


In [2]:
# simograms 
# The arguments are the following: 
# "name", "title;x axis title;yaxis title", nb of bins, min value, max value

sim_mass_2e2mu = ROOT.TH1F("sim_mass_2e2mu",";Four Lepton Invariant Mass [GeV];Events", 500, 0, 500)
sim_mass_2e2mu_b = ROOT.TH1F("sim_mass_2e2mu_b",";Four Lepton Invariant Mass [GeV];Events", 500, 0, 500)

sim_mass_2e = ROOT.TH1F("sim_mass_2e",";Electron pair Invariant Mass [GeV];Events", 500, 0, 500)
sim_mass_2mu = ROOT.TH1F("sim_mass_2mu",";Muon pair Invariant Mass [GeV];Events", 500, 0, 500)


# Zoom in the low mass regions
sim_mass_2e2mu_zoom = ROOT.TH1F("sim_mass_2e2mu_zoom",";Four Lepton Invariant Mass [GeV];Events", 500, 70, 110)
sim_mass_2e2mu_zoom_b = ROOT.TH1F("sim_mass_2e2mu_zoom_b",";Four Lepton Invariant Mass [GeV];Events", 500, 80, 100)
sim_mass_2e_zoom = ROOT.TH1F("sim_mass_2e_zoom",";Electron pair Invariant Mass [GeV];Events", 500, 80, 100)
sim_mass_2mu_zoom = ROOT.TH1F("sim_mass_2mu_zoom",";Muon pair Invariant Mass [GeV];Events", 500, 80, 100)

sim_mass_2e2mu_ZoomExtra = ROOT.TH1F("sim_mass_2e2mu_ZoomExtra",";Four Lepton Invariant Mass [GeV];Events", 500, 0, 10)
sim_mass_2e_ZoomExtra = ROOT.TH1F("sim_mass_2e_ZoomExtra",";Muon pair Invariant Mass [GeV];Events", 300, 0, 10)
sim_mass_2mu_ZoomExtra = ROOT.TH1F("sim_mass_2mu_ZoomExtra",";Muon pair Invariant Mass [GeV];Events", 500, 0, 10)

# Correlation between m(z) and m(z tilde)
# The arguments are the following:
# "name","title;x axis title;y axis title",nb x bins,min x value, max x value, nb y bins, min y value, max y value
sim_2Dcorr_real_masses = ROOT.TH2F('sim_2Dcorr_real_masses',";Mass Z boson [GeV];Mass Z' boson [GeV]",500,0,500, 100, 10, 200)
sim_2Dcorr_z_masses = ROOT.TH2F('sim_2Dcorr_z_masses',";Mass Z boson [GeV];Mass virtual Z boson [GeV]",500,0,200, 100, 10, 200)
sim_2Dcorr_zreal_4l_masses = ROOT.TH2F('sim_2Dcorr_zreal_4l_masses',";Four Lepton Invariant Mass [GeV];Mass Z boson [GeV]",500,0,500, 100, 10, 200)
sim_2Dcorr_zvirtual_4l_masses = ROOT.TH2F('sim_2Dcorr_zvirtual_4l_masses',";Four Lepton Invariant Mass [GeV];Mass virtual Z boson [GeV]",500,10,500, 100, 10, 200)

In [4]:
#Loop over entries
#invariant_2e2u = np.array([])
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   
    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            if np.linalg.norm(tree._lEta[lep]) >= 1.566 or np.linalg.norm(tree._lEta[lep]) <= 1.4442:
                n_e += 1
                Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            if np.linalg.norm(tree._lEta[lep]) >= 1.566 or np.linalg.norm(tree._lEta[lep]) <= 1.4442:
                n_p += 1
                Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            if np.linalg.norm(tree._lEta[lep]) < 2.4:
                n_mu_minus += 1
                Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            if np.linalg.norm(tree._lEta[lep]) < 2.4:
                n_mu_p +=1
                Index_mu_plus = lep

        if (n_e,n_p,n_mu_minus,n_mu_p) == (1,1,1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass4l(tree,Index_e,Index_p,Index_mu_minus,Index_mu_plus)

            invariant_mass_e = invariantmass2l(tree,Index_e,Index_p)
            invariant_mass_mu = invariantmass2l(tree,Index_mu_minus,Index_mu_plus)

            Index = np.array([Index_e,Index_p,Index_mu_plus,Index_mu_minus])

            
            #if all(np.sort([pt_e,pt_p,pt_mu_p,pt_mu_m],kind='mergesort') >= np.array([10,10,10,20])):
                
            # sim_mass_2e2mu_b.Fill(invariant_mass)
            # sim_mass_2e2mu_zoom_b.Fill(invariant_mass)
            #if Check(tree,Index) == True:
            if invariant_mass_e > 4 and invariant_mass_mu > 4:
                m1, m2 =  invariant_mass_e,invariant_mass_mu
                if m1-91 < m2-91:
                    mreal = m2
                    mvirt = m1
                else:
                    mreal = m1
                    mvirt = m2

                if mvirt>= 15:
                    sim_2Dcorr_real_masses.Fill(m1, m2)
                    sim_2Dcorr_z_masses.Fill(mvirt,mreal)
                    sim_2Dcorr_zvirtual_4l_masses.Fill(invariant_mass,mvirt)
                    sim_2Dcorr_zreal_4l_masses.Fill(invariant_mass,mreal)
                    sim_mass_2e2mu.Fill(invariant_mass)
                    sim_mass_2e2mu_zoom.Fill(invariant_mass)
                    sim_mass_2e.Fill(invariant_mass_e)
                    sim_mass_2mu.Fill(invariant_mass_mu)
            # invariant_2e2u = np.append(invariant_2e2u,invariant_mass)
            #print(invariant_mass)
            # sim_mass_2e2mu_Higgs.Fill(invariant_mass)

In [5]:
savehisto(outputfile,sim_mass_2e2mu,"sim_mass_2e2mu",logy=True)
savehisto(outputfile,sim_mass_2e,"sim_mass_2e",logy=True)
savehisto(outputfile,sim_mass_2mu,"sim_mass_2mu",logy=True)
savehisto(outputfile,sim_mass_2e2mu_b,"sim_mass_2e2mu_b",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/sim_mass_2e2mu.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/sim_mass_2e.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/sim_mass_2mu.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/sim_mass_2e2mu_b.pdf has been created
